In [1]:
import os
import yfinance as yf
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [2]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [9]:
def get_stock_data(ticker_symbol):
    """Fetches key metrics from Yahoo Finance."""
    stock = yf.Ticker(ticker_symbol)
    info = stock.info
    
    data_summary = {
        "name": info.get("longName"),
        "current_price": info.get("currentPrice"),
        "pe_ratio": info.get("forwardPE"),
        "market_cap": info.get("marketCap"),
        "52_week_high": info.get("fiftyTwoWeekHigh"),
        "52_week_low": info.get("fiftyTwoWeekLow"),
        "analyst_recommendation": info.get("recommendationKey"),
    }
    return data_summary

In [10]:
FEW_SHOT_EXAMPLES = """
<example>
Ticker: AAPL, P/E: 30, 52wk_High: 190, Current: 185

**Analysis** P/E is slightly above historical average (25), but trading near 52-week high with strong momentum from news.

**Recommendation** HOLD : The stock is near a resistance level; wait for a pullback despite strong fundamentals.
</example>

<example>
Ticker: TSLA, P/E: 75, 52wk_High: 290, Current: 150

**Analysis** Valuation is extremely high relative to peers, and price is down 48% from highs. Negative news creates further downside risk.

**Recommendation** SELL : High valuation and technical breakdown suggest further weakness.
</example>
"""

In [11]:
def get_ai_analysis(ticker, data):
    """Sends stock data to Gemini for analysis."""
    prompt = f"""
    You are a professional stock analyst. Your response will be based on the following pattern:\n{FEW_SHOT_EXAMPLES}\n
    Analyze the following data for {ticker} ({data['name']} 
    - Current Price: ${data['current_price']}
    - Forward P/E Ratio: {data['pe_ratio']}
    - 52-Week Range: ${data['52_week_low']} - ${data['52_week_high']}
    - Market Cap: {data['market_cap']}
    - General Analyst Consensus: {data['analyst_recommendation']}

    Task:
    1. Determine if the stock is a 'Buy', 'Hold', or 'Sell'.
    2. Provide a 3-sentence justification focusing on the stock's recent performance, P/E ratio, and price position.
    """
    
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=prompt
    )
    return response.text

In [12]:
user_ticker = input("Enter a stock ticker (e.g., AAPL): ").upper()
print(f"--- Fetching data for {user_ticker} ---")

try:
    stock_data = get_stock_data(user_ticker)
    analysis = get_ai_analysis(user_ticker, stock_data)
    
    print("\n--- Gemini's Analysis ---")
    print(analysis)
    print("\n**Disclaimer** This analysis is for informational purposes only and does not constitute financial advice.")
except Exception as e:
    print(f"Error: Could not retrieve data for {user_ticker}. Check the ticker symbol.", e)

--- Fetching data for NVDA ---

--- Gemini's Analysis ---
Ticker: NVDA, P/E: 24.75, 52wk_High: 212.19, Current: 187.24

**Analysis** NVDA is trading 11.7% below its 52-week high, suggesting a potential opportunity after a recent pullback. Its forward P/E of 24.75 is reasonable for a company with strong growth prospects, especially considering the general analyst consensus of 'strong buy'. The significant market cap indicates a large, established player in a high-demand sector.

**Recommendation** BUY : The stock has pulled back from its highs, the forward P/E is attractive for its growth trajectory, and the strong analyst consensus supports a bullish outlook.

**Disclaimer** This analysis is for informational purposes only and does not constitute financial advice.
